In [173]:
import pandas as pd
import matplotlib.pyplot as plt
import pdb
  

In [174]:
df = pd.read_csv("../datasets/merged_stats_n_scorecards/merged_stats_n_scorecards.csv", sep=';')
df.tail()

,red_fighter_name,blue_fighter_name,event_date,red_fighter_nickname,blue_fighter_nickname,red_fighter_result,blue_fighter_result,method,round,time,...,red_fighter_sig_str_leg_pct,blue_fighter_sig_str_leg_pct,red_fighter_sig_str_distance_pct,blue_fighter_sig_str_distance_pct,red_fighter_sig_str_clinch_pct,blue_fighter_sig_str_clinch_pct,red_fighter_sig_str_ground_pct,blue_fighter_sig_str_ground_pct,red_fighter_total_pts,blue_fighter_total_pts
2138,FELICE HERRIG,VIRNA JANDIROBA,15/08/2020,Lil Bulldog,Carcara,L,W,Submission,1,1:44,...,100,0,100,0,0,0,0,100,- - -,- - -
2139,TJ BROWN,DANNY CHAVEZ,15/08/2020,Downtown,The Colombian Warrior,L,W,Decision - Unanimous,3,5:00,...,13,36,88,81,11,0,0,18,28 28 28,29 29 29
2140,ASHLEY YODER,LIVINHA SOUZA,15/08/2020,SpiderMonkey,The Brazilian Gangsta,L,W,Decision - Unanimous,3,5:00,...,9,50,100,78,0,10,0,10,28 27 28,29 30 29
2141,CHRIS DAUKAUS,PARKER PORTER,15/08/2020,-,-,W,L,KO/TKO,1,4:28,...,10,23,78,82,16,17,5,0,- - -,- - -
2142,KAI KAMAKA,TONY KELLEY,15/08/2020,The Fighting Hawaiian,Primetime,W,L,Decision - Unanimous,3,5:00,...,0,14,78,66,19,33,1,0,29 29 29,28 28 28


# Establishing questions

### Here are a few questions I want to be answered by the end of this notebook:
1. What are the most popular locations that UFC hosts its events in?
2. What is the historical average amount of UFC fights hosted annually?
3. What are the top methods of winning?
4. What are the main fight tactics contributing to a victory?
5. Does being in a particular corner contribute to a victory? (meme, but still)
6. What fight features contribute to the fight bonuses the most?

# Data preprocessing

### Creating a single *winner* feature instead of two *red_fighter_result* and *blue_fighter_result* features

In [175]:
df.loc[:, 'winner'] = df['red_fighter_result'].apply(lambda x: 'red' if x == 'W' else 'blue')

### Changing columns from *red/blue+feature name* to *winner/loser+feature name*

In [176]:
df.head()

,red_fighter_name,blue_fighter_name,event_date,red_fighter_nickname,blue_fighter_nickname,red_fighter_result,blue_fighter_result,method,round,time,...,blue_fighter_sig_str_leg_pct,red_fighter_sig_str_distance_pct,blue_fighter_sig_str_distance_pct,red_fighter_sig_str_clinch_pct,blue_fighter_sig_str_clinch_pct,red_fighter_sig_str_ground_pct,blue_fighter_sig_str_ground_pct,red_fighter_total_pts,blue_fighter_total_pts,winner
0,ILIA TOPURIA,MAX HOLLOWAY,26/10/2024,El Matador,Blessed,W,L,KO/TKO,3,1:34,...,24,94,100,0,0,5,0,20 20 19,18 18 19,red
1,ROBERT WHITTAKER,KHAMZAT CHIMAEV,26/10/2024,The Reaper,Borz,L,W,Submission,1,3:34,...,0,100,0,0,0,0,100,- - -,- - -,blue
2,MAGOMED ANKALAEV,ALEKSANDAR RAKIC,26/10/2024,-,Rocket,W,L,Decision - Unanimous,3,5:00,...,64,90,94,9,5,0,0,29 29 29,28 28 28,red
3,LERONE MURPHY,DAN IGE,26/10/2024,The Miracle,50K,W,L,Decision - Unanimous,3,5:00,...,13,71,69,23,13,5,17,29 29 29,28 28 28,red
4,SHARA MAGOMEDOV,ARMEN PETROSYAN,26/10/2024,Bullet,Superman,W,L,KO/TKO,2,4:52,...,58,96,97,3,2,0,0,10 10 10,9 9 9,red


In [177]:
def assign_winner_n_loser(df, columns_to_change, winner_col='winner'):
    for col in columns_to_change:
        df.loc[:, f"winner_{col}"] = df[f"red_{col}"].where(df[winner_col] == 'red', df[f"blue_{col}"])
        df.loc[:, f"loser_{col}"] = df[f"red_{col}"].where(df[winner_col] == 'blue', df[f"blue_{col}"])
        
    return df 
        

In [178]:
cols_to_change = ['fighter_name', 'fighter_nickname','fighter_KD',
                  'fighter_sig_str', 'fighter_sig_str_pct', 
                  'fighter_total_str', 'fighter_TD', 'fighter_TD_pct', 
                  'fighter_sub_att', 'fighter_rev', 'fighter_ctrl', 
                  'fighter_sig_str_head', 'fighter_sig_str_body', 
                  'fighter_sig_str_leg', 'fighter_sig_str_distance', 
                  'fighter_sig_str_clinch', 'fighter_sig_str_ground', 
                  'fighter_sig_str_head_pct', 'fighter_sig_str_body_pct', 
                  'fighter_sig_str_leg_pct', 'fighter_sig_str_distance_pct', 
                  'fighter_sig_str_clinch_pct', 'fighter_sig_str_ground_pct']

df = assign_winner_n_loser(df, cols_to_change)
df.head()

,red_fighter_name,blue_fighter_name,event_date,red_fighter_nickname,blue_fighter_nickname,red_fighter_result,blue_fighter_result,method,round,time,...,winner_fighter_sig_str_body_pct,loser_fighter_sig_str_body_pct,winner_fighter_sig_str_leg_pct,loser_fighter_sig_str_leg_pct,winner_fighter_sig_str_distance_pct,loser_fighter_sig_str_distance_pct,winner_fighter_sig_str_clinch_pct,loser_fighter_sig_str_clinch_pct,winner_fighter_sig_str_ground_pct,loser_fighter_sig_str_ground_pct
0,ILIA TOPURIA,MAX HOLLOWAY,26/10/2024,El Matador,Blessed,W,L,KO/TKO,3,1:34,...,14,16,20,24,94,100,0,0,5,0
1,ROBERT WHITTAKER,KHAMZAT CHIMAEV,26/10/2024,The Reaper,Borz,L,W,Submission,1,3:34,...,33,0,0,100,0,100,0,0,100,0
2,MAGOMED ANKALAEV,ALEKSANDAR RAKIC,26/10/2024,-,Rocket,W,L,Decision - Unanimous,3,5:00,...,40,16,23,64,90,94,9,5,0,0
3,LERONE MURPHY,DAN IGE,26/10/2024,The Miracle,50K,W,L,Decision - Unanimous,3,5:00,...,23,10,7,13,71,69,23,13,5,17
4,SHARA MAGOMEDOV,ARMEN PETROSYAN,26/10/2024,Bullet,Superman,W,L,KO/TKO,2,4:52,...,44,12,18,58,96,97,3,2,0,0


### Summing scorecards from all 3 judges into one

In [179]:
df.loc[:, 'winner_fighter_total_pts'] = df['red_fighter_total_pts'].apply(lambda x: sum(int(n) for n in x.split()) if x[0].isnumeric() else "-")
df.loc[:, 'loser_fighter_total_pts'] = df['blue_fighter_total_pts'].apply(lambda x: sum(int(n) for n in x.split()) if x[0].isnumeric() else "-")

### Dropping the old features, leaving the new *winner/loser* ones

In [183]:
df = df.iloc[:, -49:]

In [184]:
df.head()

,winner,winner_fighter_name,loser_fighter_name,winner_fighter_nickname,loser_fighter_nickname,winner_fighter_KD,loser_fighter_KD,winner_fighter_sig_str,loser_fighter_sig_str,winner_fighter_sig_str_pct,...,winner_fighter_sig_str_leg_pct,loser_fighter_sig_str_leg_pct,winner_fighter_sig_str_distance_pct,loser_fighter_sig_str_distance_pct,winner_fighter_sig_str_clinch_pct,loser_fighter_sig_str_clinch_pct,winner_fighter_sig_str_ground_pct,loser_fighter_sig_str_ground_pct,winner_fighter_total_pts,loser_fighter_total_pts
0,red,ILIA TOPURIA,MAX HOLLOWAY,El Matador,Blessed,1,0,75 of 144,79 of 204,52,...,20,24,94,100,0,0,5,0,59,55
1,blue,KHAMZAT CHIMAEV,ROBERT WHITTAKER,Borz,The Reaper,0,0,3 of 4,2 of 2,75,...,0,100,0,100,0,0,100,0,-,-
2,red,MAGOMED ANKALAEV,ALEKSANDAR RAKIC,-,Rocket,0,0,55 of 114,53 of 117,48,...,23,64,90,94,9,5,0,0,87,84
3,red,LERONE MURPHY,DAN IGE,The Miracle,50K,0,1,52 of 100,46 of 106,52,...,7,13,71,69,23,13,5,17,87,84
4,red,SHARA MAGOMEDOV,ARMEN PETROSYAN,Bullet,Superman,1,0,65 of 106,77 of 140,61,...,18,58,96,97,3,2,0,0,30,27


# Data Cleaning

## NaN values

In [196]:
df.isnull().sum().sum()

np.int64(0)

## Duplicates

In [213]:
sub = df[df.duplicated()]
sub

,winner,winner_fighter_name,loser_fighter_name,winner_fighter_nickname,loser_fighter_nickname,winner_fighter_KD,loser_fighter_KD,winner_fighter_sig_str,loser_fighter_sig_str,winner_fighter_sig_str_pct,...,winner_fighter_sig_str_leg_pct,loser_fighter_sig_str_leg_pct,winner_fighter_sig_str_distance_pct,loser_fighter_sig_str_distance_pct,winner_fighter_sig_str_clinch_pct,loser_fighter_sig_str_clinch_pct,winner_fighter_sig_str_ground_pct,loser_fighter_sig_str_ground_pct,winner_fighter_total_pts,loser_fighter_total_pts
201,red,JOSEFINE KNUTSSON,JULIA POLASTRI,Thunder,Psycho,0,0,124 of 244,64 of 122,50,...,20,10,87,67,9,4,2,28,87,84
2111,red,BRIAN KELLEHER,RAY RODRIGUEZ,Boom,The Judge,0,0,1 of 3,1 of 5,33,...,100,100,100,100,0,0,0,0,-,-


In [210]:
sub.duplicated()

201     False
2111    False
dtype: bool